### Урок 2. Первые задачи. «Анализ данных о поездках на такси в Нью-Йорке»

Описание колонок:
* pickup_dt – период с точностью до часа
* pickup_month – месяц
* borough – район Нью-Йорка, из которого был сделан заказ (5 районов + аэропорт)
* pickups – число поездок за период (час)
* hday – является ли день праздничным/выходным; Y — да,  N — нет
* spd – скорость ветра в милях в час
* vsb – видимость
* temp – температура в градусах Фаренгейта
* dewp – точка росы по Фаренгейту
* slp – давление
* pcp_01 – количество осадков за час
* pcp_06 – количество осадков за 6 часов
* pcp_24 – количество осадков за 24 часа
* sd – глубина снега в дюймах

Каждая строка в датасете — запись не об одной конкретной поездке, а о всех поездках, сделанных за определенный час из того или иного района Нью-Йорка, и о том, какие были показатели погоды в этот час.

#### Задачи

##### 1) Определите, какой индекс имеет самая верхняя строка

In [8]:
import pandas as pd
taxi = pd.read_csv('taxi_nyc.csv', encoding='windows-1251', sep=',')
taxi.head(2)  #0

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp 01,pcp 06,pcp 24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


##### 2) Количество строк и столбцов в датафрейме

In [9]:
taxi.shape

(29101, 14)

##### 3) Проверить типы данных в столбцах датафрейма и определить, какой тип данных преобладает



In [11]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29101 entries, 0 to 29100
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pickup_dt     29101 non-null  object 
 1   pickup_month  29101 non-null  object 
 2   borough       26058 non-null  object 
 3   pickups       29101 non-null  int64  
 4   hday          29101 non-null  object 
 5   spd           29101 non-null  float64
 6   vsb           29101 non-null  float64
 7   temp          29101 non-null  float64
 8   dewp          29101 non-null  float64
 9   slp           29101 non-null  float64
 10  pcp 01        29101 non-null  float64
 11  pcp 06        29101 non-null  float64
 12  pcp 24        29101 non-null  float64
 13  sd            29101 non-null  float64
dtypes: float64(9), int64(1), object(4)
memory usage: 3.1+ MB


Преобладает тип данных float64

##### 4) Переименовать столбцы "pcp 01", "pcp 06", "pcp 24", заменив в них пробел на нижнее подчеркивание

In [13]:
taxi = taxi.rename(columns={'pcp 01': 'pcp_01', 'pcp 06': 'pcp_06', 'pcp 24': 'pcp_24'})
taxi.columns

Index(['pickup_dt', 'pickup_month', 'borough', 'pickups', 'hday', 'spd', 'vsb',
       'temp', 'dewp', 'slp', 'pcp_01', 'pcp_06', 'pcp_24', 'sd'],
      dtype='object')

##### 5) Определите число записей по каждому району Нью-Йорка. Сколько раз в данных встречается район Манхэттен (Manhattan)?

In [17]:
counts = taxi.borough.value_counts()
counts
#counts['Manhattan']


borough
Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
Name: count, dtype: int64

##### 6) Общее количество поездок

In [18]:
pickups_sum = taxi.pickups.sum()
pickups_sum

14265773

##### 7) Район Нью-Йорка, из которого было совершено наибольшее количество поездок

In [21]:
borough_max_pickups = taxi \
    .groupby(['borough'], as_index=False) \
    .pickups.sum() \
    .sort_values('pickups', ascending=False)
borough_max_pickups

# проверка
#borough_max_pickups_all = borough_max_pickups.pickups.sum()
#borough_max_pickups_all

,borough,pickups
3,Manhattan,10367841
1,Brooklyn,2321035
4,Queens,1343528
0,Bronx,220047
5,Staten Island,6957
2,EWR,105


##### 8) Название района с наименьшим числом поездок

In [22]:
min_pickups = taxi.groupby(['borough'])\
                .agg({'pickups': 'sum'})\
                .rename(columns={'pickups': 'sum_pickups'})\
                .idxmin()
min_pickups

sum_pickups    EWR
dtype: object

После группировки, если нет as_index=False, индексом становится поле, по которому группируем. А если полей несколько?

##### 9) Определить, из каких районов по праздникам и выходным дням в среднем поступает больше заказов, чем в обычные дни
!!! Как сделать не только глазами?

In [35]:
#taxi.hday.values  /Y N
grouped = taxi.groupby(['borough', 'hday'], as_index=False)\
                .agg({'pickups': 'mean'})
grouped

,borough,hday,pickups
0,Bronx,N,50.771073
1,Bronx,Y,48.065868
2,Brooklyn,N,534.727969
3,Brooklyn,Y,527.011976
4,EWR,N,0.023467
5,EWR,Y,0.041916
6,Manhattan,N,2401.302921
7,Manhattan,Y,2035.928144
8,Queens,N,308.899904
9,Queens,Y,320.730539


После агрегации ищем название района, в котором значение в колонке `pickups` в строке с `Y` (выходной день) больше, чем в строке с `N` (будний день).

##### 10) Определить число поездок в каждом районе по месяцам.




При группировке чтобы получилась не 1, а 3 колонки, используйется параметр `as_index` в методе `groupby`. Он определяет, что станет с группировочными колонками. Параметр принимает логическое значение (`True`/`False`) - если `as_index` равно `False`, то группировочные колонки останутся колонками в результате. А если `True` (значение по умолчанию), то группировочные колонки становятся сложным индексом (мультииндексом)


In [38]:
pickups_by_mon_bor = taxi.groupby(['borough', 'pickup_month'], as_index=False)\
                        .agg({'pickups': 'sum'})\
                        .sort_values('pickups', ascending=False)\
                        [['borough', 'pickup_month', 'pickups']]

pickups_by_mon_bor

,borough,pickup_month,pickups
21,Manhattan,Jun,1995388
23,Manhattan,May,1888800
19,Manhattan,Feb,1718571
22,Manhattan,Mar,1661261
18,Manhattan,Apr,1648278
20,Manhattan,Jan,1455543
9,Brooklyn,Jun,482466
11,Brooklyn,May,476087
6,Brooklyn,Apr,378095
10,Brooklyn,Mar,346726
